In [1]:
# Auto complete
%config IPCompleter.greedy=True

from IPython import get_ipython

import warnings
warnings.filterwarnings('ignore')

# Manipulação de dados
import pandas as pd
import numpy as np
import os # accessing directory structure

In [2]:
# PASSO 1 : defina um dataframe com as vendas dos meses fechados

In [3]:
df = pd.read_excel('vendas.xlsx')

In [4]:
df.head(3)

,ano,n_mes,grupo,produto,vendas
0,2023,1,limpeza,sabão líquido,23
1,2023,2,limpeza,sabão líquido,24
2,2023,3,limpeza,sabão líquido,30


In [5]:
# PASSO 2 : defina o último mês de vendas e o ano que deseja projetar

#MÊS
mes_fechado = df.n_mes.max()
print('útimo mês das vendas:',mes_fechado)

#ANO
ano_atual = df.ano.max()
print('ano das vendas:',ano_atual)

útimo mês das vendas: 5
ano das vendas: 2023


In [6]:
# PASSO 3 (opcional) : defina um novo dataframe do df original
# Este passo é para quando você tem um dataframe com diversos anos e meses e quer selecionar apenas o período da projeção.
# Exemplo, o df tem os anos de 2020 a 2023, mas a projeção será apenas para 2023.

# Projeção 2023
new_df = df.loc[(df["n_mes"] <= mes_fechado) & (df["ano"] == ano_atual)]

# Coluna nova indicando que o df é projeção
new_df["projeção"] = "não"

# PASSO 4: criar uma lista dos parâmetros que precisam ser interados para a projeção
# No nosso caso, o menor nível é produto, então precisamos passar pelo grupo e depois pelos prdutos
cd_grupo = new_df[["grupo"]].drop_duplicates('grupo').reset_index(drop=True)
cd_produto = new_df[["produto"]].drop_duplicates('produto').reset_index(drop=True)

# Alterar a coluna Vendas para número
new_df['vendas'] = pd.to_numeric(new_df["vendas"])

#================================#  

In [7]:
cd_grupo

,grupo
0,limpeza
1,bebidas
2,padaria


In [8]:
cd_produto

,produto
0,sabão líquido
1,detergente
2,coca-cola
3,pão
4,bolo
5,doce


In [9]:
# Final do new_df
new_df.tail(5)

,ano,n_mes,grupo,produto,vendas,projeção
25,2023,1,padaria,doce,5,não
26,2023,2,padaria,doce,8,não
27,2023,3,padaria,doce,9,não
28,2023,4,padaria,doce,4,não
29,2023,5,padaria,doce,6,não


In [10]:
# Incluindo o contador com o valor fixo
i = mes_fechado

In [11]:
# PASSO 5 # PROJEÇÃO MÉDIA MÓVEL 3 MESES
while (i < 12):
    for g in cd_grupo.grupo:
        for p in cd_produto.produto:
            df0 = new_df[(new_df['n_mes']==i) & (new_df['grupo']==g) & (new_df['produto']==p)]
            df0['n_mes'] = i+1
            df0['vendas']= round(new_df[(new_df.n_mes>=i-2) & (new_df['grupo']==g) & (new_df['produto']==p)]['vendas'].mean(),4)
            df0['projeção']='sim'
            new_df = new_df.append(df0, ignore_index=True)
        
    i=i+1

#Fim
# ============================================================= 

In [12]:
# Projeção do produto "Bolo"
new_df[new_df.produto=='bolo']

,ano,n_mes,grupo,produto,vendas,projeção
20,2023,1,padaria,bolo,10.0000,não
21,2023,2,padaria,bolo,11.0000,não
22,2023,3,padaria,bolo,19.0000,não
23,2023,4,padaria,bolo,15.0000,não
24,2023,5,padaria,bolo,14.0000,não
34,2023,6,padaria,bolo,16.0000,sim
40,2023,7,padaria,bolo,15.0000,sim
46,2023,8,padaria,bolo,15.0000,sim
52,2023,9,padaria,bolo,15.3333,sim
58,2023,10,padaria,bolo,15.1111,sim


In [ ]:
# Decompondo a projeção
# (A) # Seleciona uma linha do new_df
df0 = new_df[(new_df['n_mes']==i) & (new_df['grupo']==g) & (new_df['produto']==p)]

# (B) # Altera a coluna mês para projeção do mês seguinte
df0['n_mes'] = i+1

# (C) # Projeção para os mêses seguintes
df0['vendas']= round(new_df[(new_df.n_mes>=i-2) & (new_df['grupo']==g) & (new_df['produto']==p)]['vendas'].mean(),4)

round(numero,casas depois da vírgula) --> arredondamento de casas decimais
new_df[(new_df.n_mes>=i-2) --> selecionar os meses menor ou igual a i, resultado  [i, i-1, i-2]
new_df['grupo']==g --> seleciona um grupo conforme a iteração do for
new_df['produto']==p --> seleciona um produto conforme a iteração do for dentro do grupo
 & --> conector "e", todas as selecções anteriores dever ser verdadeiras (True)
['vendas'] --> coluna de valor resultante da seleção
.mean --> média do meses selecionados 
       
# (D) Incluindo a projeção no new_df
new_df = new_df.append(df0, ignore_index=True)


In [ ]:
#Exportar para excel
new_df.to_excel('vendas-projecao.xlsx')